In [1]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.notebook.scroll_to_top();
                jupyter.actions.call('jupyter-notebook:save-notebook');                
                
            });
        });
        
        $( document ).ready(function(){
        code_shown=false;
        $('div.input').hide()});
    
    
</script>

# Overview

This script provides the water satisification requirement index (WSRI) at a given location over time. It uses the downloaded [Famine Land Data Assimilation System](https://ldas.gsfc.nasa.gov/FLDAS/) (FLDAS)* data which can be acquired through the Crop Yield Data Download script. 

This notebook will take you step by step from the data to output.

Contributions to provide improvements are welcome.

\**McNally, A., Arsenault, K., Kumar, S., Shukla, S., Peterson, P., Wang, S., Funk, C., Peters-Lidard, C.D., & Verdin, J. P. (2017). A land data assimilation system for sub-Saharan Africa food and water security applications. Scientific Data, 4, 170012*

## User Tips

- **Every cell ran when the notebook opened**


- **Each input is "live"; entering an input then running the cell will delete your input**


- **After making an input each cell below that input needs to be rerun to use that new input**


- **If you get an error rerun the cells above the cell that errored and it will rectify**

## 0: Python Dependencies 

This cell imports the python dependencies we use to download the data. It has already run, but if click the run cell button, the buttons for this cell will disappear. 

In [2]:
#import libraries
from toggle_code import toggle_code as hide_code
from toggle_code import run_code as run_code

import glob 
from netCDF4 import Dataset
import pandas as pd
import numpy as np2
import math
import pcse
import datetime
import ipywidgets as widgets
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import Spectral4
from bokeh.models import Legend, BoxAnnotation, Toggle
from bokeh.layouts import layout, gridplot, column
tile_provider = get_provider('STAMEN_TERRAIN')
#create pyproj transformer to convert form lat/long to web mercator
from pyproj import Transformer
transformer = Transformer.from_crs('epsg:4326','epsg:3857')
#from IPython.display import HTML
output_notebook()
import warnings
warnings.filterwarnings("ignore", message="Cannot find a last shown plot to update.")


Loading BokehJS ...

## 1: Initializing the data

Next, we store the max and min dates from our list of months, so we can create a table (dataframe) with an index for the climate data from each month. We also add the column variables that we want to extract to the dataframe. 

Now we have an empty dataframe to fill with the downloaded data for the variables we need to calculate different crop growths

*If you want to look at all the possible variables to extract from the FLDAS dataset you can open the code and delete the # in front of the print(data.variables.keys()) line below.* 

In [3]:
hide_code()
run_code()
#Retrieve the dates form the downlaoded files to serves as an index for the dataframe
all_months = []
#print("The following dates were downloaded:")
#Retrieves dates to create index list form downloaded files
for file in glob.glob(r'data/*.nc4'):
    #print(file)
    data = Dataset(file, 'r')
    #time = data.variables['time'].units
    #print(time)
    date = file[26:30] + '-' + file[30:32]
    #month = time[11:21]
    all_months.append(date)
    #print(time)
    #print (date)

#Retrieve elevevation data and stores in a dictionary of
# Key = Long, Lat pair
# Value = Elevation
elev_dict = {}
for file in glob.glob(r'data/*.csv'):
    elev = pd.read_csv(file)
    for idx, row in elev.iterrows():
        elev_dict[(row["Latitude"],row["Longitude"])] =row["Elevation"]


# sort months of python list
all_months.sort( )

month_start = min(all_months)
month_end = max(all_months)
#print("Start:", month_start)
#print("Month End:", month_end)
date_range = pd.date_range(start = month_start, end = month_end, freq = 'MS' )

#print(data.variables.keys())

#creates an empty data frame with the dates
df = pd.DataFrame(0.0, columns = ['air_temp' , 'humidity' , 'net_short_radiation', 'net_long_radiation', 'wind_speed', 'evapotranspiration'], index = date_range)
df.index.name = 'Date'

df


air_temp  humidity  net_short_radiation  net_long_radiation  \
Date                                                                      
2019-01-01       0.0       0.0                  0.0                 0.0   
2019-02-01       0.0       0.0                  0.0                 0.0   
2019-03-01       0.0       0.0                  0.0                 0.0   
2019-04-01       0.0       0.0                  0.0                 0.0   
2019-05-01       0.0       0.0                  0.0                 0.0   
2019-06-01       0.0       0.0                  0.0                 0.0   
2019-07-01       0.0       0.0                  0.0                 0.0   
2019-08-01       0.0       0.0                  0.0                 0.0   
2019-09-01       0.0       0.0                  0.0                 0.0   
2019-10-01       0.0       0.0                  0.0                 0.0   
2019-11-01       0.0       0.0                  0.0                 0.0   
2019-12-01       0.0       0.0                  0.0                 0.0   
2020-01-01       0.0       0.0                  0.0                 0.0   
2020-02-01       0.0       0.0                  0.0                 0.0   
2020-03-01       0.0       0.0                  0.0                 0.0   
2020-04-01       0.0       0.0                  0.0                 0.0   
2020-05-01       0.0       0.0                  0.0                 0.0   
2020-06-01       0.0       0.0                  0.0                 0.0   
2020-07-01       0.0       0.0                  0.0                 0.0   
2020-08-01       0.0       0.0                  0.0                 0.0   
2020-09-01       0.0       0.0                  0.0                 0.0   
2020-10-01       0.0       0.0                  0.0                 0.0   
2020-11-01       0.0       0.0                  0.0                 0.0   

            wind_speed  evapotranspiration  
Date                                        
2019-01-01         0.0                 0.0  
2019-02-01         0.0                 0.0  
2019-03-01         0.0                 0.0  
2019-04-01         0.0                 0.0  
2019-05-01         0.0                 0.0  
2019-06-01         0.0                 0.0  
2019-07-01         0.0                 0.0  
2019-08-01         0.0                 0.0  
2019-09-01         0.0                 0.0  
2019-10-01         0.0                 0.0  
2019-11-01         0.0                 0.0  
2019-12-01         0.0                 0.0  
2020-01-01         0.0                 0.0  
2020-02-01         0.0                 0.0  
2020-03-01         0.0                 0.0  
2020-04-01         0.0                 0.0  
2020-05-01         0.0                 0.0  
2020-06-01         0.0                 0.0  
2020-07-01         0.0                 0.0  
2020-08-01         0.0                 0.0  
2020-09-01         0.0                 0.0  
2020-10-01         0.0                 0.0  
2020-11-01         0.0                 0.0

## 2. Get the specific location to assess 

Here is where you can input your desired location to calculate the Water Requirements Satisficiation Index and Crop Yield over time. 

In [4]:
hide_code()
run_code()
file_loc = r"data/FLDAS_NOAH01_C_GL_M.A" + all_months[0][0:4]+all_months[0][5:7] + ".001.nc.SUB.nc4"
data_loc =Dataset(file_loc, 'r')
lat_array = data_loc.variables['Y'][:]#.compressed() # unmasks numpy_array lat stored in the FLDAS files 
long_array = data_loc.variables['X'][:]#.compressed() # unmasks numpy array long stored in FLDAS files
min_lat = lat_array[0]
max_lat = lat_array[-1]
min_long = long_array[0]
max_long = long_array[-1]
long_points = {'globe':[], 'web':[]}
lat_points = {'globe':[], 'web':[]}
for i in range(len(lat_array)):
    for j in range(len(long_array)):
        if round(lat_array[i],2) not in lat_points['globe']:
            lat_points['globe'].append(round(lat_array[i],2))
        if round(long_array[j],2) not in long_points['globe']:
            long_points['globe'].append(round(long_array[j],2))
        pt = transformer.transform(lat_array[i], long_array[j])
        if pt[0] not in long_points['web']: 
            long_points['web'].append(pt[0])
        if pt[1] not in lat_points['web']:
            lat_points['web'].append(pt[1])
            
pts = [(min_lat, min_long), (max_lat, max_long)]
bbox = []
for pt in transformer.itransform(pts): 
    bbox.append(pt)       
p = figure(x_range=(bbox[0][0], bbox[1][0]),y_range=(bbox[0][1], bbox[1][1]),x_axis_type="mercator", y_axis_type="mercator")
#add the map form the Bokeh map vendor in this case Stamen_Terrain --- see documentation
p.add_tile(tile_provider)
y = [lat_points["web"][50]]
x = [long_points["web"][80]]
loc = p.square(x,y, color= "red", size = 10, alpha = 0.5)


def update(latitude, longitude):
    idx_long = long_points["globe"].index(longitude)
    idx_lat = lat_points["globe"].index(latitude)
    loc.data_source.data['x'] = [long_points["web"][idx_long]]
    loc.data_source.data['y'] = [lat_points["web"][idx_lat]]
    push_notebook()
    return longitude, latitude
    
loc_input = interact(update,latitude=(lat_points["globe"][0], lat_points["globe"][-1]), 
                     longitude=(long_points["globe"][0],long_points["globe"][-1]))

show(p, notebook_handle=True)
print ("  ") #Hides the output

interactive(children=(FloatSlider(value=17.55, description='latitude', max=23.75, min=11.35), FloatSlider(valu…

# 3. Knowing the Data

The next several cells show the data from the selected location and transform it so we can assess crop yields. 

In [13]:
hide_code()
run_code()
# Get user input form sliders
lat_user = loc_input.__dict__["widget"].children[0].__dict__["_trait_values"]["value"]
lon_user = loc_input.__dict__["widget"].children[1].__dict__["_trait_values"]["value"]
#Fill dataframe with data from FLDAS files
df_time_index = 0
for month in all_months:

    file_name = r"data/FLDAS_NOAH01_C_GL_M.A" + month[0:4]+ month[5:7] + ".001.nc.SUB.nc4"
    data = Dataset(file_name, 'r')
    #print(file_name)
    
    # storing the lat and lon data into variables of netCDF file
    lat = data.variables['Y'][:]
    lon = data.variables['X'][:]
    
    #squared diff of lat and lon
    sq_diff_lat = (lat - lat_user)**2
    sq_diff_lon = (lon - lon_user)**2
    
    #getindex of minimum sq difference
    min_index_lat =sq_diff_lat.argmin()
    min_index_lon = sq_diff_lon.argmin()
    
    #accessing the soil moisture data and storing it into the final dataframe
    air_temp = data.variables['Tair_f_tavg']
    humidity = data.variables['Qair_f_tavg']
    net_short_radiation = data.variables['SWdown_f_tavg']
    net_long_radiation = data.variables['Lwnet_tavg']
    wind_speed = data.variables['Wind_f_tavg']
    evapotranspiration = data.variables['Evap_tavg']


    #possibly creating the time range for each month and each iteration
    #start = month
    #d_range = pd.date
   # df.iloc[df_time_index] = SM40[0,min_index_lat, min_index_lon]
    df.air_temp[df_time_index] = air_temp[0,min_index_lat, min_index_lon]
    df.humidity[df_time_index] = humidity[0,min_index_lat, min_index_lon]
    df.net_short_radiation[df_time_index] = net_short_radiation[0,min_index_lat, min_index_lon]
    df.net_long_radiation[df_time_index] = net_long_radiation[0,min_index_lat, min_index_lon]
    df.wind_speed[df_time_index] = wind_speed[0,min_index_lat, min_index_lon]
    df.evapotranspiration[df_time_index] = evapotranspiration[0,min_index_lat, min_index_lon]


    df_time_index += 1



df

air_temp  humidity  net_short_radiation  net_long_radiation  \
Date                                                                        
2019-01-01  293.899445  0.002330           241.499115         -119.827629   
2019-02-01  295.192535  0.002129           264.408478         -123.316399   
2019-03-01  301.694305  0.003097           271.832550         -121.345497   
2019-04-01  305.065765  0.003615           294.075562         -133.402649   
2019-05-01  306.518616  0.010453           264.089905         -102.920174   
2019-06-01  305.133789  0.011866           276.281494          -95.945694   
2019-07-01  302.580597  0.013876           253.015625          -78.712395   
2019-08-01  299.490631  0.015433           233.931900          -62.593636   
2019-09-01  301.082520  0.014063           236.496674          -74.099358   
2019-10-01  300.805145  0.012219           225.819794          -81.739761   
2019-11-01  300.095032  0.005138           245.850052         -120.597153   
2019-12-01  293.414734  0.002898           237.400024         -130.209869   
2020-01-01  292.066223  0.003005           226.695831         -113.378616   
2020-02-01  294.331238  0.002612           257.288818         -125.563057   
2020-03-01  301.640717  0.003297           286.807526         -130.392166   
2020-04-01  305.747253  0.005762           278.183289         -122.197243   
2020-05-01  305.385376  0.009775           271.129517         -109.003784   
2020-06-01  304.502197  0.012806           274.429138          -99.074814   
2020-07-01  300.467651  0.015071           245.169907          -71.568718   
2020-08-01  299.338440  0.015819           233.552643          -61.295551   
2020-09-01  299.830109  0.014934           231.841324          -67.996223   
2020-10-01  300.414978  0.009914           247.218658          -99.641190   
2020-11-01  296.828888  0.005605           247.116257         -113.535881   

            wind_speed  evapotranspiration    VP        WR       WRSI  
Date                                                                   
2019-01-01    7.109678            0.000004   5.7  5.798230   0.104227  
2019-02-01    7.321100            0.000004   5.7  6.587709   0.067762  
2019-03-01    7.560431            0.000003   6.7  8.369192   0.040583  
2019-04-01    6.414071            0.000002   8.5  8.386663   0.072802  
2019-05-01    4.631352            0.000002  13.2  7.435077   0.389809  
2019-06-01    4.958045            0.000019  15.7  6.517200   4.808620  
2019-07-01    5.048081            0.000029  18.4  5.877076  16.979024  
2019-08-01    4.746761            0.000035  20.4  4.137020  54.172942  
2019-09-01    3.428925            0.000029  17.4  5.817277  18.252930  
2019-10-01    4.636156            0.000015  12.3  7.552238   2.246248  
2019-11-01    6.378662            0.000006   8.1  7.142583   0.453319  
2019-12-01    8.370172            0.000004   6.3  6.811180   0.191876  
2020-01-01    8.982525            0.000003   5.7  6.400845   0.100426  
2020-02-01    8.927598            0.000004   5.7  7.372793   0.061539  
2020-03-01    6.838092            0.000003   6.7  8.180990   0.036113  
2020-04-01    4.825009            0.000002   8.5  9.292443   0.024022  
2020-05-01    5.277357            0.000001  13.2  8.013834   0.119260  
2020-06-01    4.803023            0.000010  15.7  8.021225   5.478073  
2020-07-01    5.039196            0.000032  18.4  4.981372  18.458521  
2020-08-01    4.878229            0.000037  20.4  4.324020  49.012338  
2020-09-01    3.794797            0.000034  17.4  5.367696  22.970130  
2020-10-01    4.492451            0.000012  12.3  6.747607   2.030936  
2020-11-01    7.178642            0.000006   8.1  6.645448   0.553572

## 4. Converting Units

We are using the [Penman-Monteith model (PM model)](https://en.wikipedia.org/wiki/Penman%E2%80%93Monteith_equation#cite_note-1) to calculate the seasonal crop water requirement. In order to input the variables we need to convert the units from the FLDAS data to units utilized by the Penman-Monteith model. 

In the following table we change: 
- evapotranspiration from kilograms per second to millimeter per day
- wind from 10 meter elevation to 2 meter elevation
- temperature from Kelvin to Celsius

In [14]:
hide_code()
run_code()
#converting units to correct measures for the PM equation
# converts kg/m^2/s to mm/day
def convert_to_mm_day(eot):
    return eot * 86400
df['evapotranspiration'] = df['evapotranspiration'].apply(convert_to_mm_day)

# converts wind speed at 10m elevation to wind speed at a 2m elevation
def convert_wind(speed):
    return speed * 4.87 / math.log ( 67.8 * 10 - 5.42 ) 
df['wind_speed'] = df['wind_speed'].apply(convert_wind)


#converts Kelvin to Celcius
def convert_to_C(temp):
    return temp -273.15
df['air_temp'] = df['air_temp'].apply(convert_to_C)
df

air_temp  humidity  net_short_radiation  net_long_radiation  \
Date                                                                       
2019-01-01  20.749445  0.002330           241.499115         -119.827629   
2019-02-01  22.042535  0.002129           264.408478         -123.316399   
2019-03-01  28.544305  0.003097           271.832550         -121.345497   
2019-04-01  31.915765  0.003615           294.075562         -133.402649   
2019-05-01  33.368616  0.010453           264.089905         -102.920174   
2019-06-01  31.983789  0.011866           276.281494          -95.945694   
2019-07-01  29.430597  0.013876           253.015625          -78.712395   
2019-08-01  26.340631  0.015433           233.931900          -62.593636   
2019-09-01  27.932520  0.014063           236.496674          -74.099358   
2019-10-01  27.655145  0.012219           225.819794          -81.739761   
2019-11-01  26.945032  0.005138           245.850052         -120.597153   
2019-12-01  20.264734  0.002898           237.400024         -130.209869   
2020-01-01  18.916223  0.003005           226.695831         -113.378616   
2020-02-01  21.181238  0.002612           257.288818         -125.563057   
2020-03-01  28.490717  0.003297           286.807526         -130.392166   
2020-04-01  32.597253  0.005762           278.183289         -122.197243   
2020-05-01  32.235376  0.009775           271.129517         -109.003784   
2020-06-01  31.352197  0.012806           274.429138          -99.074814   
2020-07-01  27.317651  0.015071           245.169907          -71.568718   
2020-08-01  26.188440  0.015819           233.552643          -61.295551   
2020-09-01  26.680109  0.014934           231.841324          -67.996223   
2020-10-01  27.264978  0.009914           247.218658          -99.641190   
2020-11-01  23.678888  0.005605           247.116257         -113.535881   

            wind_speed  evapotranspiration    VP        WR       WRSI  
Date                                                                   
2019-01-01    5.317691            0.308101   5.7  5.798230   0.104227  
2019-02-01    5.475824            0.313050   5.7  6.587709   0.067762  
2019-03-01    5.654832            0.265116   6.7  8.369192   0.040583  
2019-04-01    4.797411            0.202517   8.5  8.386663   0.072802  
2019-05-01    3.464025            0.197001  13.2  7.435077   0.389809  
2019-06-01    3.708375            1.604254  15.7  6.517200   4.808620  
2019-07-01    3.775718            2.503562  18.4  5.877076  16.979024  
2019-08-01    3.550345            3.008853  20.4  4.137020  54.172942  
2019-09-01    2.564668            2.464961  17.4  5.817277  18.252930  
2019-10-01    3.467618            1.322047  12.3  7.552238   2.246248  
2019-11-01    4.770927            0.523195   8.1  7.142583   0.453319  
2019-12-01    6.260479            0.362207   6.3  6.811180   0.191876  
2020-01-01    6.718489            0.294379   5.7  6.400845   0.100426  
2020-02-01    6.677407            0.320231   5.7  7.372793   0.061539  
2020-03-01    5.114558            0.268407   6.7  8.180990   0.036113  
2020-04-01    3.608871            0.178847   8.5  9.292443   0.024022  
2020-05-01    3.947205            0.123942  13.2  8.013834   0.119260  
2020-06-01    3.592426            0.886291  15.7  8.021225   5.478073  
2020-07-01    3.769072            2.779231  18.4  4.981372  18.458521  
2020-08-01    3.648677            3.238319  20.4  4.324020  49.012338  
2020-09-01    2.838323            2.929540  17.4  5.367696  22.970130  
2020-10-01    3.360133            1.072457  12.3  6.747607   2.030936  
2020-11-01    5.369273            0.497078   8.1  6.645448   0.553572

## 5. Elevation data

We then look up the elevation for our location from the downloaded elevation data for our desired location as an input for the Penman-Monteith model algorithm. 

In [15]:
hide_code()
run_code()
#gets elevation in meters using google elevation api
#might need to square diff it

elevation = elev_dict[(lat_user,lon_user)]
print ("Elevation:", elevation, "meters")


#Adding in dates, months and Vapor pressure
dfvp = pd.DataFrame(0.0, columns = [], index = date_range)
dfvp.index.name = 'Date'
dfvp["dates"] =  pd.date_range(start = month_start, end = month_end, freq = 'MS' ) 
dfvp['month'] = dfvp['dates'].dt.month 

vp = [5.7, 5.7, 6.7, 8.5, 13.2, 15.7, 18.4, 20.4, 17.4, 12.3, 8.1, 6.3]
def months(x , vp):
        return vp[x-1]

dfvp['VP'] = dfvp.apply(lambda row: months(row.month , vp) , axis=1)
df["VP"] = dfvp["VP"]


Elevation: 445.0 meters


## 6. Water Requirement Calculator 

Next, we use the [Python Crop Simulation Environment](https://pcse.readthedocs.io/en/stable/) and specifically the Penman-Monteith algorithm to calculate the Water Requirement Satsification Index (WRSI) for each month in the dataframe adding the WR (water requirement on the far right of the dataframe). 

In [16]:
hide_code()
run_code()
#Calculating WR
df["dates"] = dfvp["dates"]
#pcse.util.penman_monteith(DAY, LAT, ELEV, TMIN, TMAX, AVRAD, VAP, WIND2)
df['WR'] = df.apply(lambda row:  pcse.util.penman_monteith(row.dates, lat_user, elevation, row.air_temp , row.air_temp , row.net_short_radiation, row.VP, row.wind_speed), axis=1)
del df["dates"]
df

air_temp  humidity  net_short_radiation  net_long_radiation  \
Date                                                                       
2019-01-01  20.749445  0.002330           241.499115         -119.827629   
2019-02-01  22.042535  0.002129           264.408478         -123.316399   
2019-03-01  28.544305  0.003097           271.832550         -121.345497   
2019-04-01  31.915765  0.003615           294.075562         -133.402649   
2019-05-01  33.368616  0.010453           264.089905         -102.920174   
2019-06-01  31.983789  0.011866           276.281494          -95.945694   
2019-07-01  29.430597  0.013876           253.015625          -78.712395   
2019-08-01  26.340631  0.015433           233.931900          -62.593636   
2019-09-01  27.932520  0.014063           236.496674          -74.099358   
2019-10-01  27.655145  0.012219           225.819794          -81.739761   
2019-11-01  26.945032  0.005138           245.850052         -120.597153   
2019-12-01  20.264734  0.002898           237.400024         -130.209869   
2020-01-01  18.916223  0.003005           226.695831         -113.378616   
2020-02-01  21.181238  0.002612           257.288818         -125.563057   
2020-03-01  28.490717  0.003297           286.807526         -130.392166   
2020-04-01  32.597253  0.005762           278.183289         -122.197243   
2020-05-01  32.235376  0.009775           271.129517         -109.003784   
2020-06-01  31.352197  0.012806           274.429138          -99.074814   
2020-07-01  27.317651  0.015071           245.169907          -71.568718   
2020-08-01  26.188440  0.015819           233.552643          -61.295551   
2020-09-01  26.680109  0.014934           231.841324          -67.996223   
2020-10-01  27.264978  0.009914           247.218658          -99.641190   
2020-11-01  23.678888  0.005605           247.116257         -113.535881   

            wind_speed  evapotranspiration    VP        WR       WRSI  
Date                                                                   
2019-01-01    5.317691            0.308101   5.7  6.508978   0.104227  
2019-02-01    5.475824            0.313050   5.7  7.060159   0.067762  
2019-03-01    5.654832            0.265116   6.7  9.185707   0.040583  
2019-04-01    4.797411            0.202517   8.5  8.872799   0.072802  
2019-05-01    3.464025            0.197001  13.2  6.578321   0.389809  
2019-06-01    3.708375            1.604254  15.7  6.049741   4.808620  
2019-07-01    3.775718            2.503562  18.4  4.811815  16.979024  
2019-08-01    3.550345            3.008853  20.4  3.234925  54.172942  
2019-09-01    2.564668            2.464961  17.4  3.513866  18.252930  
2019-10-01    3.467618            1.322047  12.3  5.274818   2.246248  
2019-11-01    4.770927            0.523195   8.1  7.370174   0.453319  
2019-12-01    6.260479            0.362207   6.3  6.716587   0.191876  
2020-01-01    6.718489            0.294379   5.7  6.676458   0.100426  
2020-02-01    6.677407            0.320231   5.7  7.544929   0.061539  
2020-03-01    5.114558            0.268407   6.7  8.602553   0.036113  
2020-04-01    3.608871            0.178847   8.5  7.463208   0.024022  
2020-05-01    3.947205            0.123942  13.2  6.893838   0.119260  
2020-06-01    3.592426            0.886291  15.7  5.740712   5.478073  
2020-07-01    3.769072            2.779231  18.4  4.129025  18.458521  
2020-08-01    3.648677            3.238319  20.4  3.240580  49.012338  
2020-09-01    2.838323            2.929540  17.4  3.445297  22.970130  
2020-10-01    3.360133            1.072457  12.3  5.063249   2.030936  
2020-11-01    5.369273            0.497078   8.1  6.776508   0.553572

## 7. Calculating final WRSI (Water Requirement Satisfaction Index)

We use evapotranspiration and WR to calculate the final Water Requirement Satisfaction Index for each month

In [17]:
hide_code()
run_code()
#Calculate final WRSI
df['WRSI'] = df.apply(lambda row:  100* (row.evapotranspiration/row.WR), axis=1)
df
     

air_temp  humidity  net_short_radiation  net_long_radiation  \
Date                                                                       
2019-01-01  20.749445  0.002330           241.499115         -119.827629   
2019-02-01  22.042535  0.002129           264.408478         -123.316399   
2019-03-01  28.544305  0.003097           271.832550         -121.345497   
2019-04-01  31.915765  0.003615           294.075562         -133.402649   
2019-05-01  33.368616  0.010453           264.089905         -102.920174   
2019-06-01  31.983789  0.011866           276.281494          -95.945694   
2019-07-01  29.430597  0.013876           253.015625          -78.712395   
2019-08-01  26.340631  0.015433           233.931900          -62.593636   
2019-09-01  27.932520  0.014063           236.496674          -74.099358   
2019-10-01  27.655145  0.012219           225.819794          -81.739761   
2019-11-01  26.945032  0.005138           245.850052         -120.597153   
2019-12-01  20.264734  0.002898           237.400024         -130.209869   
2020-01-01  18.916223  0.003005           226.695831         -113.378616   
2020-02-01  21.181238  0.002612           257.288818         -125.563057   
2020-03-01  28.490717  0.003297           286.807526         -130.392166   
2020-04-01  32.597253  0.005762           278.183289         -122.197243   
2020-05-01  32.235376  0.009775           271.129517         -109.003784   
2020-06-01  31.352197  0.012806           274.429138          -99.074814   
2020-07-01  27.317651  0.015071           245.169907          -71.568718   
2020-08-01  26.188440  0.015819           233.552643          -61.295551   
2020-09-01  26.680109  0.014934           231.841324          -67.996223   
2020-10-01  27.264978  0.009914           247.218658          -99.641190   
2020-11-01  23.678888  0.005605           247.116257         -113.535881   

            wind_speed  evapotranspiration    VP        WR       WRSI  
Date                                                                   
2019-01-01    5.317691            0.308101   5.7  6.508978   4.733474  
2019-02-01    5.475824            0.313050   5.7  7.060159   4.434035  
2019-03-01    5.654832            0.265116   6.7  9.185707   2.886177  
2019-04-01    4.797411            0.202517   8.5  8.872799   2.282450  
2019-05-01    3.464025            0.197001  13.2  6.578321   2.994693  
2019-06-01    3.708375            1.604254  15.7  6.049741  26.517727  
2019-07-01    3.775718            2.503562  18.4  4.811815  52.029460  
2019-08-01    3.550345            3.008853  20.4  3.234925  93.011517  
2019-09-01    2.564668            2.464961  17.4  3.513866  70.149551  
2019-10-01    3.467618            1.322047  12.3  5.274818  25.063368  
2019-11-01    4.770927            0.523195   8.1  7.370174   7.098816  
2019-12-01    6.260479            0.362207   6.3  6.716587   5.392723  
2020-01-01    6.718489            0.294379   5.7  6.676458   4.409211  
2020-02-01    6.677407            0.320231   5.7  7.544929   4.244320  
2020-03-01    5.114558            0.268407   6.7  8.602553   3.120080  
2020-04-01    3.608871            0.178847   8.5  7.463208   2.396377  
2020-05-01    3.947205            0.123942  13.2  6.893838   1.797873  
2020-06-01    3.592426            0.886291  15.7  5.740712  15.438703  
2020-07-01    3.769072            2.779231  18.4  4.129025  67.309637  
2020-08-01    3.648677            3.238319  20.4  3.240580  99.930210  
2020-09-01    2.838323            2.929540  17.4  3.445297  85.030120  
2020-10-01    3.360133            1.072457  12.3  5.063249  21.181199  
2020-11-01    5.369273            0.497078   8.1  6.776508   7.335310

## 8. Crop specific Water Requirements

Finally, we calculate a general WRSI. However, we can find the crop specific WRSI data based on how much water a crop requires at diffent stages of its growth. We used the crop co-efficients from the online [United Nations Food and Agricultural Guidelines for Computing Crop Water Requirements](http://www.fao.org/3/X0490e/x0490e0b.htm#crop%20coefficients). 

### Please pick your crops and their life cycle(s) of interest.

* Plant - Water requirements at planting
* Grow - Water requirements during growth
* Harvest - Water requirements at harvest time

(Shift for group of selections; Ctrl-Shift for multiple individual selections)

In [19]:
hide_code()
run_code()
crop_human = widgets.SelectMultiple(
    options = ["cowpeas : plant", "cowpeas : grow", "cowpeas : harvest", 
               "spring wheat : plant", "spring wheat : grow", "spring wheat : harvest",
              "ground nut : plant", "ground nut : grow", "ground nut : harvest", 
              "maize : plant", "maize : grow", "maize : harvest", 
              "millet : plant", "millet : grow", "millet : harvest",
              "sorghum : plant", "sorghum : grow", "sorghum : harvest"],
    description='Crops',
    disabled=False
)

crop_selected = []
def update(crops):
    crop_selected = list(crops)
    return crop_selected

crops_selection = interact(update, crops= crop_human)


grazing = widgets.SelectMultiple(
    options = ["rotated grazing : grow", "rotated grazing : harvest",
               "extensive grazing : grow", "extensive grazing : harvest"], 
    description="Grazing", 
    disabled = False)


graze = []
def g_update(grazing):
    graze_selected = list(grazing)
    return graze_selected
    
graze_selection = interact(g_update, grazing = grazing)

interactive(children=(SelectMultiple(description='Crops', options=('cowpeas : plant', 'cowpeas : grow', 'cowpe…

interactive(children=(SelectMultiple(description='Grazing', options=('rotated grazing : grow', 'rotated grazin…

In [20]:
hide_code()
run_code()

crops = list(crops_selection.__dict__["widget"].children[0].__dict__["_trait_values"]["value"])
grazes = list(graze_selection.__dict__["widget"].children[0].__dict__["_trait_values"]["value"])

crop_dict = {"cowpeas":[0.4, 1.05,0.6],
            "maize": [0.7,1.20, .35], 
             "millet":[.7,1.20, .35], 
             "sorghum":[.7, 1.20, .35], 
             "spring wheat": [0.4, 1.15,0.33], 
             "ground nut": [0.4, 1.15, 0.6], 
             "rotated grazing": [None, 1.05, 0.85],
             "extensive grazing": [None,0.75,0.75]}

#cowpeas,spring wheat, ground nut and grazing have variable requirements...figure out how to deal with that. 

reqs = {}

def parse_crops(food):
    '''
    param: list of user inputs from crop and gazing selection 
    output: dictionary with crop and lifecycle times
    '''
   
    
    for f in food: 
        inputs = f.split(':')
        crop = inputs[0][0:-1] # get rid of final space
        cycle = inputs[1][1:] #get rid of lead space
                    
        if crop not in reqs.keys(): 
            reqs[crop] =[cycle]
        else: 
            reqs[crop].append(cycle)
    
    return reqs

reqs = parse_crops(crops)
reqs = parse_crops(grazes)


#Initializing empty data frame
dfcrops = pd.DataFrame(0.0, columns = [], index = date_range)
dfcrops.index.name = 'Date'

def get_cycle(cycle):
    cycle_idx = ""
    if cycle == "plant":
        cycle_idx = 0
    elif cycle == "grow":
        cycle_idx = 1
    else: 
        cycle_idx = 2
    return cycle_idx

def make_crop_df(reqs):
    
    for crop,cycle in reqs.items(): 
        for c in cycle: 
            idx = get_cycle(c)
            dfcrops[crop+" " + c] = df["WRSI"]/crop_dict[crop][idx]

make_crop_df(reqs)
dfcrops

maize plant  maize grow  maize harvest
Date                                              
2019-01-01     6.762105    3.944561      13.524210
2019-02-01     6.334336    3.695029      12.668671
2019-03-01     4.123111    2.405148       8.246221
2019-04-01     3.260643    1.902042       6.521287
2019-05-01     4.278133    2.495578       8.556267
2019-06-01    37.882467   22.098106      75.764934
2019-07-01    74.327801   43.357884     148.655601
2019-08-01   132.873595   77.509597     265.747191
2019-09-01   100.213644   58.457959     200.427287
2019-10-01    35.804811   20.886140      71.609623
2019-11-01    10.141166    5.915680      20.282333
2019-12-01     7.703890    4.493936      15.407781
2020-01-01     6.298873    3.674343      12.597746
2020-02-01     6.063314    3.536933      12.126629
2020-03-01     4.457257    2.600067       8.914514
2020-04-01     3.423396    1.996981       6.846792
2020-05-01     2.568390    1.498228       5.136781
2020-06-01    22.055290   12.865586      44.110581
2020-07-01    96.156625   56.091364     192.313250
2020-08-01   142.757444   83.275175     285.514887
2020-09-01   121.471601   70.858434     242.943201
2020-10-01    30.258856   17.650999      60.517712
2020-11-01    10.479015    6.112759      20.958030

## Plotting Data

Now we can plot the final WRSI(Seasonal Water Requirement Satisfaction Index) models for each crop/crop stage. The WRSI model calculates the crop yield in relation to a water deficit. It follows the scale

100+: Very Good

95-100: Good

80-94: Average

60-79: Mediocre

50-59: Poor

In [21]:
hide_code()
run_code()

columns = list(dfcrops.columns)

plots = widgets.SelectMultiple(
    options = columns, 
    description="Data to Plot", 
    disabled = False)

title = widgets.Text(
    value='Crop Yields',
    description='Title',
    disabled=False
)

font = widgets.IntText(
    value=10,
    description='Font Size',
    disabled=False
)

size = widgets.IntText(
    value=600,
    description='Plot Size',
    disabled=False
)

to_plot = []
def plot_update(plots, title, font, size):
    to_plot = list(plots)
    font_label = str(font)+"pt"
    axis_label = str(font*1.5) + "pt"
    title_label = str(font*2)+"pt"
    leg = []
    ph = size
    if len(to_plot) == 0:
        pass
    else: 
        above = 0
        p = figure(plot_width=size, plot_height=ph, title=title, x_range = list(dfcrops.index.astype(str)))
        p.xaxis.major_label_orientation = 45
        p.title.text_font_size = title_label
        p.xaxis.axis_label_text_font_size = font_label
        p.xaxis.axis_label = "Date"
        p.yaxis.axis_label = "Water Satisfication"
        for plot, color in zip(to_plot, Spectral4): 
            l = p.line(list(dfcrops.index.astype(str)), dfcrops[plot], color = color, line_width = 2)
            if max(dfcrops[plot]) > above:
                above = max(dfcrops[plot])
            leg.append((plot, [l]))
        legend = Legend(items= leg, location=(0,ph/2))
        green_box = BoxAnnotation(top=above+10, bottom=94, fill_color='green', fill_alpha=0.2)
        yellow_box = BoxAnnotation(top=94, bottom=80, fill_color='yellow', fill_alpha=0.2)
        orange_box = BoxAnnotation(top=80, bottom=50, fill_color='orange', fill_alpha=0.2)
        red_box = BoxAnnotation(top=50, bottom=0, fill_color='red', fill_alpha=0.2)
        boxes = [green_box, yellow_box, orange_box, red_box]
        p.add_layout(legend, "right")
       
        for box in boxes: 
            p.add_layout(box)
                
        toggle1 = Toggle(label="Good Water", button_type="success", active=True)
        toggle1.js_link('active', green_box, 'visible')
        
        toggle2 = Toggle(label= "Average Water", button_type="success", active=True)
        toggle2.js_link('active', yellow_box, 'visible')
        
        toggle3 = Toggle(label="Mediocre Water", button_type="success", active=True)
        toggle3.js_link('active', orange_box, 'visible')
        
        toggle4 = Toggle(label="Poor Water", button_type="success", active=True)
        toggle4.js_link('active', red_box, 'visible')
        #inputs = column(plots, title, font, size)
        show(layout([p], [toggle1, toggle2], [toggle3, toggle4]))
            

plot_selection = interact(plot_update, plots = plots, title = title, font = font, size = size)

interactive(children=(SelectMultiple(description='Data to Plot', options=('maize plant', 'maize grow', 'maize …